In [1]:
import openai
import os

# Set your API key
api_key = os.getenv("OPENAI_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key


In [26]:
def generate_profile_description(user_description):
    response = openai.Completion.create(
        engine="davinci-002",
        prompt=f"Generate a user profile description: {user_description}",
        max_tokens=50,  # Adjust the token limit as needed
        n=1,  # Number of responses
        stop=None,  # Stop tokens (optional)
    )
    return response.choices[0].text.strip()

def generate_matching_criteria(user_profile, investor_profile):
    # Generate matching criteria using LLM
    criteria = []
    criteria.append(generate_profile_description(user_profile["Industry"]))
    criteria.append(generate_profile_description(", ".join(user_profile["Verticals"])))
    criteria.append(generate_profile_description(user_profile["Geography"]))
    criteria.append(generate_profile_description(user_profile["Investment Stage"]))
    criteria.append(generate_profile_description(str(user_profile["Planned Raise Amount"])))
    criteria.append(generate_profile_description(user_profile["Stage of development"]))
    criteria.append(generate_profile_description(user_profile["Revenue/profitability"]))
    criteria.append(generate_profile_description(user_profile["Fundraising history"]))
    criteria.append(generate_profile_description(user_profile["Intellectual property"]))
    criteria.append(generate_profile_description(user_profile["Competitive landscape"]))
    criteria.append(generate_profile_description(user_profile["Customer Base (# paying clients)"]))
    criteria.append(generate_profile_description(user_profile["Growth potential"]))
    
    # Combine generated criteria into a single text
    startup_description = "\n".join(criteria)

    criteria = []
    criteria.append(generate_profile_description(investor_profile["Fund from Fund Master"]))
    criteria.append(generate_profile_description(investor_profile["Target Customer"]))
    criteria.append(generate_profile_description(investor_profile["Industry Final"]))
    criteria.append(generate_profile_description(investor_profile["Vertical Final"]))
    criteria.append(generate_profile_description(investor_profile["Investor Type"]))
    criteria.append(generate_profile_description(investor_profile["Preferred Geography (from Fund from Fund Master)"]))
    criteria.append(generate_profile_description(investor_profile["Preferred Investment Stage"]))
    criteria.append(generate_profile_description(investor_profile["Preferred Investment Type"]))
    criteria.append(generate_profile_description(investor_profile["Min check size"]))
    criteria.append(generate_profile_description(investor_profile["Max Check size"]))
    criteria.append(generate_profile_description(investor_profile["Impact Investor"]))
    criteria.append(generate_profile_description(investor_profile["Number of investments (Portfolio Size)"]))
    criteria.append(generate_profile_description(investor_profile["Return Expectations"]))
    # criteria.append(generate_profile_description(row["Exit Strategy Preference"]))
    criteria.append(generate_profile_description(investor_profile["Network and Support Resources "]))
    # criteria.append(generate_profile_description(row["Due diligence process"]))
    criteria.append(generate_profile_description(investor_profile["Decision-making timeline"]))
    criteria.append(generate_profile_description(investor_profile["Reputation"]))
    criteria.append(generate_profile_description(investor_profile["Portfolio Success Track Record (% Exits) "]))
    criteria.append(generate_profile_description(investor_profile["Managers/LPs Turnover"]))
    criteria.append(generate_profile_description(investor_profile["% Portfolio Diversity"]))
    investor_description = "\n".join(criteria)
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=f"""
                    ROLE: Assume you are Broker that has expertise in VC, investment and managing funds
                    and you are working on investors meet new founders that will profit them \n
                    TASK: Generate matching criteria among a founder and investor, importing matching criteria that should somewhat overlap are - 
                    Industry, Verticals, Geography,Planned Raise Amount and other key factors that a investor will consider. 
                    For example: A investor looking to invest in healthcare would look for a founder builing in the same and even if the 
                    Industry is not matching, verticals should match to a certain extent. 
                    Similary for Planned raised amount should be in between of min check size and max check size of investor(min check of 0.25 means 0.25 million dollars i.e. 250000 dollars, and this applies to max check size too). 
                    Please take into account many other key aspects that you as an expert broker will take \n
                    On the basis of above context generate matching criteria:\n\nStartup Profile:\n{startup_description}\n\nInvestor Profile:\n{investor_description}""",
        max_tokens=120,  # Adjust the token limit as needed
        n=1,  # Number of responses
        stop=None,  # Stop tokens (optional)
    )
    matching_criteria = response.choices[0].text.strip()
    return matching_criteria

def generate_matching_score(matching_criteria):
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=f"""ROLE: Assume you are Broker that has expertise in VC, investment and managing funds 
                    and you are working on investors meet new founders that will profit them \n
                    TASK: Generate matching criteria among a founder and investor, importing matching criteria that should somewhat overlap are - 
                    Industry, Verticals, Geography,Planned Raise Amount and other key factors that a investor will consider. 
                    For example: A investor looking to invest in healthcare would look for a founder builing in the same and even if the 
                    Industry is not matching, verticals should match to a certain extent. 
                    Similary for Planned raised amount should be in between of min check size and max check size of investor(min check of 0.25 means 0.25 million dollars i.e. 250000 dollars, and this applies to max check size too).
                    Please take into account many other key aspects that you as an expert broker will take \n
                    On the basis of above context calculate matching score: {matching_criteria}""",
        max_tokens=10,  # Adjust the token limit as needed
        n=1,  # Number of responses
        stop=None,  # Stop tokens (optional)
    )
    score = float(response.choices[0].text.strip())  # Assuming LLM returns a numerical score
    return score


In [15]:
import csv

founders_data = []
with open('data/founder_v2.csv', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)
    for row in reader:
        founders_data.append(dict(zip(headers, row)))

founder_document = founders_data[10]

In [22]:
import csv

investors_data = []
with open('data/inv_clean_v1.csv', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)
    for row in reader:
        investors_data.append(dict(zip(headers, row)))

inv_document = investors_data[0]

In [23]:
inv_document

{'\ufeffInvestor Name': 'Maarten Goossens',
 'Fund from Fund Master': 'Anterra Capital',
 'Target Customer': 'B2B',
 'Industry Final': 'Agriculture & Forestry,Energy and Climate,Food and Beverage,Healthcare',
 'Vertical Final': 'AgTech,Climate Tech,Digital Health,HealthTech,Beauty & Wellness',
 'Investor Type': 'Venture Capital',
 'Website': 'www.anterracapital.com',
 'Preferred Geography (from Fund from Fund Master)': 'Asia, Canada, Europe, United States',
 'Preferred Investment Stage': 'Pre-SEED,Series A,SEED',
 'Preferred Investment Type': 'Convertible Debt,SAFE,Debt',
 'Min check size': '$0.10',
 'Max Check size': '$2.00',
 'HQ Location': 'Amsterdam, Netherlands',
 'URL (from Fund from Fund Master)': 'www.anterracapital.com',
 'Preferred TAM': '$5B to $10B',
 'Impact Investor': 'No',
 'Number of investments (Portfolio Size)': '90',
 'Return Expectations': '2X - 5X',
 'Exit Strategy Preference': 'Acquisition,IPO',
 'Network and Support Resources ': 'Heavily Supportive',
 'Due dilige

In [25]:
matching_criteria = generate_matching_criteria(founder_document, inv_document)

KeyError: 'Min check size(Minimum investment they are looking to make in Million dollars)'